In [21]:
import pandas as pd
import numpy as np
import re


In [22]:

df = pd.read_csv("./data/WELFake_Dataset.csv")
print(df.columns)
print(df.shape)
print(type(df.loc[7,"text"]))

text_iterator = df['text']
title_iterator = df['title']

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')
(72134, 4)
<class 'str'>


Tokenise

In [23]:
def process_text(text_iterator):
    text_processed = []
    for text in text_iterator:        
        try:
            # remove punctuation
            t = re.sub("[^a-zA-Z0-9]", " ", text)
            # remove multiple spaces
            t = re.sub(r" +", " ", t)
            # remove newline
            t = re.sub(r"\n", " ", t)
            # clear trailing whitespaces
            t = t.strip()
            # lowercase
            t = t.lower()
            # tokenise
            t = t.split(" ")
            # drop empty string
            t = list(filter(lambda x: x != "", t))
            if len(t) == 0:
                text_processed.append(None)
            else:
                text_processed.append(t)
        except:
            text_processed.append(None)
            continue
    return text_processed


text_processed = process_text(text_iterator)
title_processed = process_text(title_iterator)

print(len(text_processed))
print(len(title_processed))
print(text_processed[2])

72134
72134
['now', 'most', 'of', 'the', 'demonstrators', 'gathered', 'last', 'night', 'were', 'exercising', 'their', 'constitutional', 'and', 'protected', 'right', 'to', 'peaceful', 'protest', 'in', 'order', 'to', 'raise', 'issues', 'and', 'create', 'change', 'loretta', 'lynch', 'aka', 'eric', 'holder', 'in', 'a', 'skirt']


Remove Stop Words

In [24]:
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


def remove_stop_words_inner(text, stop_words):
    no_stop_words = []
    if text is None:
        return None
    for token in text:
        if token not in stop_words:
            no_stop_words.append(token)
    return no_stop_words


def remove_stop_words_outer(text_list, stop_words):
    for index, text in enumerate(text_list):
        t = remove_stop_words_inner(text, stop_words)
        text_list[index] = t
    return text_list


text_no_stop_words = remove_stop_words_outer(text_processed, stop_words)
title_no_stop_words = remove_stop_words_outer(title_processed, stop_words)
print(text_no_stop_words[2])
print(len(text_no_stop_words))
print(len(title_no_stop_words))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/limjohn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['demonstrators', 'gathered', 'last', 'night', 'exercising', 'constitutional', 'protected', 'right', 'peaceful', 'protest', 'order', 'raise', 'issues', 'create', 'change', 'loretta', 'lynch', 'aka', 'eric', 'holder', 'skirt']
72134
72134


Create new table

In [25]:
df_output = pd.DataFrame(columns=["index", "title", "text", "class"])
df_output["title"] = title_no_stop_words
df_output["text"] = text_no_stop_words
df_output["index"] = np.arange(len(title_no_stop_words))
df_output["class"] = df["label"]
df_output.to_csv("./data/clean_data.csv")

As title and text contribute differently to the prediction. so we should create two types of table, we will inherit the index so we can match the title and texts. 

In [26]:
df_text = df_output.drop(columns="title")
df_text = df_text[df_text["text"].notnull()]
df_title = df_output.drop(columns="text")
df_title = df_title[df_title["title"].notnull()]

df_text.to_pickle('./data/clean_text.pkl')
df_text.to_csv('./data/clean_text.csv')
df_title.to_pickle('./data/clean_title.pkl')
df_title.to_csv('./data/clean_title.csv')